Load all the csvs into dataframs and analyse.

In [ ]:
import pandas as pd
from matchday_demand_forecasting.config import RAW_DATA_DIR, RANDOM_SEED

pd.set_option('display.max_columns', None)

files = ['ORDER_DETAILS.xlsx', 'ORDER_HEADER.xlsx']
data_paths = [RAW_DATA_DIR / file for file in files]

df_1 = pd.read_excel(data_paths[0])
df_2 = pd.read_excel(data_paths[1])

2025-09-26 18:43:54.947 | INFO     | matchday_demand_forecasting.config:<module>:11 - PROJ_ROOT path is: /Users/muhammadalisyed/Desktop/Work/Learning/Machine Learning/Projects/matchday_demand_forecasting
/opt/anaconda3/envs/matchday_demand_forecasting/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
